In [1]:
import sys
sys.path.insert(1, '/home/yannik/vssil')

import torch

from src.utils.kpt_utils import get_image_patches, kpts_2_img_coordinates
from src.losses.kpt_tracking_metric import kpt_tracking_metric
from src.losses.kpt_visual_metric import kpt_visual_metric
from contrastive_loss_test_data.test_keypoints import get_perfect_keypoints, get_bad_keypoints, get_random_keypoints
from contrastive_loss_test_data.test_data import load_sample_images

sample_size = 4
batch_size = 16
patch_size = (16, 16)
n_bins = 100

In [2]:
torch.manual_seed(123)

# Load example image frames
img_tensor = load_sample_images(sample_size=sample_size,
                                path="/home/yannik/vssil/contrastive_loss_test_data/990000.mp4").unsqueeze(0)
print(img_tensor.mean())
img_tensor = img_tensor.repeat((batch_size, 1, 1, 1, 1))
N, T, C, H, W = img_tensor.shape

tensor(0.2376)


In [3]:
# Load example key-points
perfect_kpt_coordinates = get_perfect_keypoints(T=sample_size).unsqueeze(0)
perfect_kpt_coordinates = perfect_kpt_coordinates.repeat((batch_size, 1, 1, 1))

bad_kpt_coordinates = get_bad_keypoints(T=sample_size).unsqueeze(0)
bad_kpt_coordinates = bad_kpt_coordinates.repeat((batch_size, 1, 1, 1))

random_kpt_coordinates = get_random_keypoints(T=sample_size).unsqueeze(0)
random_kpt_coordinates = random_kpt_coordinates.repeat((batch_size, 1, 1, 1))

In [4]:
M_track_perfect = kpt_tracking_metric(perfect_kpt_coordinates, img_tensor, patch_size, n_bins)
M_track_bad = kpt_tracking_metric(bad_kpt_coordinates, img_tensor, patch_size, n_bins)
M_track_random = kpt_tracking_metric(random_kpt_coordinates, img_tensor, patch_size, n_bins)
print(M_track_perfect)
print(M_track_bad)
print(M_track_random)

tensor(49.9524)
tensor(57.)
tensor(211.1429)


In [5]:
M_vis_perfect_color, M_vis_perfect_grad = kpt_visual_metric(perfect_kpt_coordinates, img_tensor, patch_size, n_bins)
M_vis_bad_color, M_vis_bad_grad = kpt_visual_metric(bad_kpt_coordinates, img_tensor, patch_size, n_bins)
M_vis_random_color, M_vis_random_grad = kpt_visual_metric(random_kpt_coordinates, img_tensor, patch_size, n_bins)
print(M_vis_perfect_color)
print(M_vis_bad_color)
print(M_vis_random_color)
print()
print(M_vis_perfect_grad)
print(M_vis_bad_grad)
print(M_vis_random_grad)

tensor(0.2859)
tensor(0.0278)

tensor(2.5600)
tensor(2.5600)

tensor(0.2751)
tensor(0.0269)

tensor(2.5600)
tensor(2.5600)

tensor(0.2259)
tensor(0.0114)

tensor(2.5600)
tensor(2.5600)

tensor(54.0714)
tensor(51.3333)
tensor(127.7262)

tensor(54.0714)
tensor(51.3333)
tensor(127.7262)


In [6]:
print(torch.relu(M_track_perfect - M_vis_perfect_color + 50.0))
print(torch.relu(M_track_bad - M_vis_bad_color + 50.0))
print(torch.relu(M_track_random - M_vis_random_color + 50.0))

tensor(45.8810)
tensor(55.6667)
tensor(133.4167)
